In [1]:
import torch
from torch.utils.data import DataLoader, random_split
import numpy as np
from utils import calculate_kurtosis
import torch.optim as optim
from model import Encoder, Decoder
import torch.nn as nn
import torch.nn.functional as F
import zarr
from data.tdms_to_npy_scaling import patch_matrix
import tqdm

In [2]:
dataset = torch.load('autoencoder/data/dataset_p1500_filtered.pt')
print(f"Dataset shape: {dataset.shape}")
# Define the dataset sizes
num_samples = len(dataset)
train_size = int(0.7 * num_samples)
val_size = int(0.1 * num_samples)
test_size = num_samples - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader instances
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

Dataset shape: torch.Size([31158, 1500, 32])


In [3]:
encod_f = Encoder()
encod_m = Encoder()

optimizer = optim.Adam(list(encod_f.parameters()) + list(encod_m.parameters()), lr=0.001, weight_decay=0.001)
num_epochs = 100


In [4]:
class KurtosisLoss(nn.Module):
    def __init__(self, alpha=1.0, beta=1.0, gamma=10.0, epsilon=0.01):
        super(KurtosisLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.epsilon = epsilon

    def forward(self, k_f, k_m):
        # Loss to maximize k_f and k_m
        k_f = torch.abs(k_f)
        k_m = torch.abs(k_m)

        k_f = k_f.view(-1)
        k_m = k_m.view(-1)
        maximize_kf_km = -self.alpha * (k_f + k_m)
        
        # Penalty to ensure k_m > k_f
        penalty = self.gamma * torch.relu(k_f - k_m + self.epsilon)
        
        # Total loss
        loss = maximize_kf_km  + penalty
        return loss.sum()

criterion = KurtosisLoss(alpha=1.0, beta=1.0, gamma=10.0, epsilon=0.01)
for epoch in range(num_epochs):
    encod_f.train()
    encod_m.train()

    train_loss = 0.0
    # Replace the regular for loop with tqdm.notebook.tqdm (for Jupyter notebooks) or tqdm.tqdm (for regular Python scripts)
    for patches in tqdm.tqdm(train_dataloader):
        optimizer.zero_grad()

        # Forward pass through encoders
        encoded_fetal = encod_f(patches)
        encoded_maternal = encod_m(patches)

        k_f = calculate_kurtosis(encoded_fetal)
        k_m = calculate_kurtosis(encoded_maternal)

        loss = criterion(k_f, k_m)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation step
    encod_f.eval()
    encod_m.eval()
    val_loss = 0.0
    with torch.no_grad():
        # Replace the regular for loop with tqdm.notebook.tqdm (for Jupyter notebooks) or tqdm.tqdm (for regular Python scripts)
        for patches in tqdm.tqdm(val_dataloader):
            encoded_fetal = encod_f(patches)
            encoded_maternal = encod_m(patches)
            k_f = calculate_kurtosis(encoded_fetal)
            k_m = calculate_kurtosis(encoded_maternal)
            loss = criterion(k_f, k_m)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_dataloader):.4f}, Val Loss: {avg_val_loss:.4f}')
    
# Testing step
encod_f.eval()
encod_m.eval()
test_loss = 0.0
with torch.no_grad():
    for patches in test_dataloader:
        encoded_fetal = encod_f(patches)
        encoded_maternal = encod_m(patches)
        k_f = calculate_kurtosis(encoded_fetal)
        k_m = calculate_kurtosis(encoded_maternal)
        loss = criterion(k_f, k_m)
        test_loss += loss.item()

print(f'Test Loss: {test_loss/len(test_dataloader):.4f}')

100%|██████████| 49/49 [00:00<00:00, 60.98it/s]


Epoch [1/100], Train Loss: 21167.3589, Val Loss: 36820.0327


100%|██████████| 49/49 [00:00<00:00, 67.57it/s]


Epoch [2/100], Train Loss: 20434.6211, Val Loss: 28499.4083


100%|██████████| 49/49 [00:00<00:00, 66.32it/s]


...

Epoch [48/100], Train Loss: 21462.8025, Val Loss: -769.7218


100%|██████████| 49/49 [00:00<00:00, 67.89it/s]


Epoch [49/100], Train Loss: 20555.4276, Val Loss: 33397.5520


 83%|████████▎ | 284/341 [00:06<00:01, 43.74it/s]


KeyboardInterrupt: 